In [7]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
This is a module to exemplarise an adapter class used in creating a 
BioCypher-compatible graph database from pypath objects. It is called from
the "create_and_update_db.py" script in this directory.

Copyright 2021, Heidelberg University Clinic

File author(s): Dénes Türei
                Sebastian Lobentanzer
                ...

Distributed under GPLv3 license, see LICENSE.txt.
"""

import json
import os
import yaml
import importlib as imp

import biocypher

import pypath.core.network as pypath_network
import pypath.resources.network as pypath_netres
import pypath.share.session as _session


class BiocypherAdapter(_session.Logger):
    """
    The connection can be defined in three ways:
        * Providing a ready ``neo4j.Driver`` instance
        * By URI and authentication data
        * By a YML config file

    Args:
        driver (neo4j.Driver): A ``neo4j.Driver`` instance, created by,
            for example, ``neo4j.GraphDatabase.driver``.
        db_name (str): Name of the database (Neo4j graph) to use.
        db_uri (str): Protocol, host and port to access the Neo4j server.
        db_auth (tuple): Neo4j server authentication data: tuple of user
            name and password.
        config_file (str): Path to a YML config file which provides the URI,
            user name and password.
        network (pypath.core.network.Network): A network database object.
        wipe (bool): Wipe the database after connection, ensuring the data
            is loaded into an empty database.
    """


    def __init__(
        self,
        driver = None,
        db_name = 'neo4j',
        db_uri = 'bolt://localhost:7687',
        db_auth = ('neo4j', '1234'),
        config_file = '/config/module_config.yaml',
        network = None,
        wipe = False,
    ):

        _session.Logger.__init__(self, name = 'bcy_adapter')

        self.bcy = biocypher.Driver(
            driver = driver,
            db_name = db_name,
            db_uri = db_uri,
            db_auth = db_auth,
            config_file = config_file,
            wipe = wipe
        )

        if network:

            self.set_network(network)


    def reload(self):
        """
        Reloads the object from the module level.
        """

        modname = self.__class__.__module__
        mod = __import__(modname, fromlist = [modname.split('.')[0]])
        imp.reload(mod)
        new = getattr(mod, self.__class__.__name__)
        setattr(self, '__class__', new)


    def set_network(self, network):

        self._log('Added new network.')
        self.network = network


    def build_python_object(self):
        """
        Builds a network database with two datasets: 'pathway' and
        'mirna_target'. Intended to be an example. The dataset is preloaded
        to avoid waiting time when applying multiple database tests.
        The resulting network database is stored under the :py:attr:`network`
        attribute.
        """
        
        with open('CROssBAR_Web-service_Example_1.json', 'r') as file:
            crossbar_graph = json.load(file)        
        
        self.set_network(crossbar_graph)

    def translate_python_object_to_neo4j(self, network = None):
        """
        Loads a pypath network into the biocypher (Neo4j) backend.

        Args:
            - network (pypath.core.network.Network): A network database 
              object. If `None`, the value of :py:attr:`network` will be 
              used.
        """

        network = network or self.network
        if not network:

            self._log('No network provided.')
            return

        
        def gen_nodes(nodes):
            for n in nodes:
                if len(n['data']) == 3:
                    id_ = self._process_id(n['data']['id'])
                    type_ = str(n['data']['Node_Type']).replace("_N","").capitalize()
                    props = {'display_name': str(n['data']['display_name'])}
                    #print(id_, type_, props)
                    yield (id_, type_, props)
                
                elif len(n['data']) == 4:
                    id_ = self._process_id(n['data']['id'])
                    type_ = str(n['data']['Node_Type']).replace("_N","").capitalize()
                    
                    if type_.startswith("Kegg_"):
                        type_ = type_.replace("Kegg_","").capitalize()
                    if type_ == "Hpo":
                        type_ = "Phenotype"
                    if type_ == "Prediction":
                        type_ = "Compound"
                    
                    props = {'display_name': str(n['data']['display_name']), 'enrich_score': str(n['data']['enrichScore'])}
                    #print(id_, type_, props)
                    yield (id_, type_, props)
                
                    
        
        id_type_tuples = gen_nodes(list(network["nodes"]))
        #print(next(id_type_tuples))
        self.bcy.add_nodes(id_type_tuples)
       
        def gen_edges(edges):
            types_dict = {'interacts w/':'Interacts_With', 'is associated w/':'Is_Associated_With', 'is related to':'Is_Related_To',
                         'targets':'Targets', 'is involved in':'Is_Involved_In', 'indicates':'Indicates', 'modulates':'Modulates'}
            for e in edges:
                source = self._process_id(e["data"]["source"])
                target = self._process_id(e["data"]["target"])
                type_ = types_dict[str(e["data"]["label"])]
                props = {"Edge_Type": e["data"]["Edge_Type"]}
                #print(source, target, type_, props)
                yield (source, target, type_, props)
        
        src_tar_type_tuples = gen_edges(list(network["edges"]))
        self.bcy.add_edges(src_tar_type_tuples)
        

    def write_to_csv_for_admin_import(self, network = None, db_name=None):
        """
        Loads a pypath network into the biocypher (Neo4j) backend using
        the fast Admin Import function, which requires text files that 
        need to be properly formatted since it turns off safety measures
        at import.

        Args:
            - network (pypath.core.network.Network): A network database 
              object. If `None`, the value of :py:attr:`network` will be 
              used.
        """

        network = network or self.network

        if not network:
            self._log("No network provided.")
            return

        # write nodes
        def gen_nodes(nodes):
            for n in nodes:
                id = self._process_id(n.identifier)
                type = n.entity_type
                props = {"taxon": n.taxon, "label": n.label}
                print(props)
                yield (id, type, props)
        id_type_tuples = gen_nodes(network.nodes.values())

        self.bcy.write_nodes(id_type_tuples, db_name=db_name)

        # write edges
        def gen_edges(edges):
            for e in edges:
                src = self._process_id(e.id_a)
                tar = self._process_id(e.id_b)
                type = e.type
                props = {"effect": e.effect, "directed": e.directed}
                yield (src, tar, type, props)
        src_tar_type_tuples = gen_edges(network.generate_df_records())

        self.bcy.write_edges(src_tar_type_tuples, db_name=db_name)

        self.bcy.write_import_call()


    def _process_id(self, identifier):
        """
        Replace critical symbols in ids so that neo4j doesn't throw
        a type error.
        """
        identifier = str(identifier)
        
        """
        replace_characters = [':', '-']
        
        for character in replace_characters:            
            if character in identifier:
                identifier = identifier.replace(character, "_")
        """
        
        return identifier


    def load(self, obj):
        """
        Loads any compatible object into the biocypher (Neo4j) database.

        Args:
            obj: An object from this module compatible with the current
                adapter. Currently the following database objects are
                supported:
                    * :py:class:`pypath.core.network.Network`
        """

        if hasattr(obj, 'nodes') and hasattr(obj, 'interactions'):

            self.translate_python_object_to_neo4j(network = obj)

In [8]:
adapt = BiocypherAdapter(wipe=True)

In [9]:
adapt.build_python_object()

In [10]:
delete = 'MATCH (n) DETACH DELETE n'

from biocypher.driver import Driver

d = Driver(db_name = 'neo4j', db_uri = 'bolt://localhost:7687', db_auth = ('neo4j', '1234'), increment_version=False)
d.query(delete)

([], <neo4j.work.summary.ResultSummary at 0x1ce1e6828c8>)

In [11]:
adapt.translate_python_object_to_neo4j()